In [24]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.utils import resample


from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from tabulate import tabulate



In [25]:


class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, num_iteration=10000, regularization='l2', lambda_=0.01):
        self.learning_rate = learning_rate
        self.num_iteration = num_iteration
        self.regularization = regularization
        self.lambda_ = lambda_ 
        self.theta = None
        self.bias = None
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, h, y):
        emperical_loss = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
        complexity_penalty = 0

        if self.regularization == 'l2':
            complexity_penalty = self.lambda_ * (self.theta ** 2).mean()
        elif self.regularization == 'l1':
            complexity_penalty = self.lambda_ * abs(self.theta).mean()
        else:
            complexity_penalty = 0

        return emperical_loss + complexity_penalty 

    
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        
        self.theta = np.zeros(X.shape[1])

        
        self.bias = 0


        # print("--------------------------------------")
        for i in range(self.num_iteration):
            z = np.dot(X, self.theta) + self.bias
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            gradient_bias = np.sum(h - y) / y.size
            
            if self.regularization == 'l2':
                self.theta -= self.learning_rate * (gradient + self.lambda_ * self.theta)
            elif self.regularization == 'l1':
                self.theta -= self.learning_rate * (gradient + self.lambda_ * np.sign(self.theta))
            else:
                self.theta -= self.learning_rate * gradient
            
            self.bias -= self.learning_rate * gradient_bias

            if i % 1000 == 0:
                z = np.dot(X, self.theta) + self.bias
                h = self.__sigmoid(z)
                # print(f'loss: {self.__loss(h, y)} \t')
        
    
    def predict(self, X):
        X = np.array(X)
        z = np.dot(X, self.theta) + self.bias
        return np.array([1 if i > 0.5 else 0 for i in self.__sigmoid(z)])
    
    def predict_proba(self, X):
        X = np.array(X)
        z = np.dot(X, self.theta) + self.bias
        return self.__sigmoid(z)

In [26]:

def load_dataset(dataset_path):
    # Load the datset
    dataset = pd.read_csv(dataset_path)
    return dataset 

In [27]:
def split_features_target(dataframe, target_column):
    X = dataframe.drop(target_column, axis=1)
    y = dataframe[target_column]
    return X, y 

In [28]:
def replace_numeric_missing_values(dataframe):
    numerical_cols = dataframe.select_dtypes(include=['number']).columns
    dataframe[numerical_cols] = dataframe[numerical_cols].fillna(dataframe[numerical_cols].mean())
    return dataframe
    

In [29]:
def replace_categorical_missing_values(dataframe):
    non_numerical_cols = dataframe.select_dtypes(exclude=['number']).columns
    for col in non_numerical_cols:
        dataframe[col] = dataframe[col].fillna(dataframe[col].mode()[0])
    return dataframe

In [30]:
def remove_rows_with_missing_target(dataframe, target_column):
    dataframe = dataframe.dropna(subset=[target_column])
    return dataframe

In [31]:
def convert_non_numeric_to_categotical(features):
    non_numeric_columns = features.select_dtypes(exclude=['number']).columns
    for column in non_numeric_columns:
        features[column] = pd.Categorical(features[column])
    return features

In [32]:
def perform_encoding(features):
    # if there are two uinque values in a column, then perform label encoding
    # if there are more than two unique values in a categorical column, then perform one hot encoding
    non_categorical_columns = []
    for column in features.columns:
        if features[column].dtype.name == 'category':
            if len(features[column].unique()) == 2:
                features[column] = labelEncoder.fit_transform(features[column])
        else:
            non_categorical_columns.append(column)
            
    features = pd.get_dummies(features).astype('int64')

    return features, non_categorical_columns

In [33]:
def scale_features(features, non_categorical_columns, scaling_strategy='Standard'):
    if scaling_strategy == 'Standard':
        scaler = StandardScaler()
    elif scaling_strategy == 'MinMax':
        scaler = MinMaxScaler()
    else:
        return "Invalid Scaling Strategy"
    
    
    features[non_categorical_columns] = scaler.fit_transform(features[non_categorical_columns])
    return features
    

In [34]:
def top_n_features_based_on_correlation(features, target, n):
    correlation = features.corrwith(target)
    correlation = correlation.sort_values(ascending=False, key=lambda x: abs(x))
    # print(correlation)
    
    if(n > len(correlation)):
        n = len(correlation)

    selected_features = correlation.head(n).index
    features_selected = features[selected_features]

    return features_selected

In [35]:
def preprocess_dataset(dataframe, target_column, scaling_strategy='Standard', top_n_features=50, columns_to_scale=None):

    print("Dataframe shape: ", dataframe.shape)
    print()

    print("Before removing rows with missing target values: ", dataframe.shape)
    # Remove rows with missing target values
    dataframe =  remove_rows_with_missing_target(dataframe, target_column)
    print("After removing rows with missing target values: ", dataframe.shape)
    print()

    print("Before dropping duplicate rows: ", dataframe.shape)
    # Drop duplicate rows
    dataframe.drop_duplicates(inplace=True)
    print("After dropping duplicate rows: ", dataframe.shape)
    print()

    print("Before replacing numeric null values: ", dataframe.isnull().sum().sum())
    # Replace missing values in the numeric columns with the mean 
    dataframe = replace_numeric_missing_values(dataframe)
    print("After replacing numeric null values ", dataframe.isnull().sum().sum())
    print()

    print("Before replacing categorical null values: ", dataframe.isnull().sum().sum())
    # Replace missing values in the categorical columns with the mode
    dataframe = replace_categorical_missing_values(dataframe)
    print("After replacing categorical null values: ", dataframe.isnull().sum().sum())
    print()


    # Split the dataset into features and target
    features, target = split_features_target(dataframe, target_column)

    

    print("Before encoding: ", features.shape)  

    # Label Encode the target column
    target = labelEncoder.fit_transform(target)
    
    # convert non-numeric columns to categorical
    features = convert_non_numeric_to_categotical(features)
    
    # Perform encoding
    # Save the non-categorical columns for scaling
    features, non_categorical_columns = perform_encoding(features)

    print("After encoding: ", features.shape)
    print()

    if(columns_to_scale is not None):
        non_categorical_columns = columns_to_scale
    
    # Scale the features
    features = scale_features(features, non_categorical_columns, scaling_strategy)
    

    target = pd.DataFrame(target, columns=[target_column])
    target_series = target[target_column]

    
    # Select top n features based on correlation
    features = top_n_features_based_on_correlation(features, target_series, top_n_features)

    return features, target_series

In [36]:
def calculate_score(y_test, y_pred, pred_proba):
    # Calculate Accuracy Sensitivity Specificity Precision F1-score AUROC AUPR

    accuracy = accuracy_score(y_test, y_pred)

    sensitivity = recall_score(y_test, y_pred)

    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    specificity = tn / (tn + fp)

    precision = precision_score(y_test, y_pred, zero_division=1)

    f1 = f1_score(y_test, y_pred)

    auroc = roc_auc_score(y_test, pred_proba)

    aupr = average_precision_score(y_test, pred_proba)

    return accuracy, sensitivity, specificity, precision, f1, auroc, aupr
    


In [37]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

def draw_violin_plots(base_metrics):
    # Convert the base_metrics dictionary to a DataFrame for easy plotting with seaborn
    metrics_df = pd.DataFrame({
        'Accuracy': base_metrics['accuracy'],
        'Sensitivity': base_metrics['sensitivity'],
        'Specificity': base_metrics['specificity'],
        'Precision': base_metrics['precision'],
        'F1-score': base_metrics['f1'],
        'AUROC': base_metrics['auroc'],
        'AUPR': base_metrics['aupr']
    })

    # Melt the DataFrame into long format suitable for seaborn
    metrics_df_melted = metrics_df.melt(var_name="Metric", value_name="Score")


    plt.figure(figsize=(10, 6))
    

    colors = sns.color_palette("husl", len(metrics_df.columns))

    
    sns.violinplot(x="Metric", y="Score", hue="Metric", data=metrics_df_melted, palette=colors, legend=False)

    plt.title("Performance Metrics", fontsize=16)

  
    plt.tight_layout()
    plt.show()


In [38]:
# calculate the mean and standard deviation of the metrics
def calculate_mean_std(base_metrics):
    mean_std = {}
    for metric in base_metrics:
        metric_values = np.array(base_metrics[metric])
        mean_std[metric] = {'mean': metric_values.mean(), 'std': metric_values.std()}
    return mean_std

In [39]:
def stacking_ensemble(features, target, val_size=0.2, test_size=0.2, n_base_classifiers=9, lr=0.01, num_iteration=10000, regularization='l2', lambda_=0.01, features_train=None, target_train=None, features_test=None, target_test=None):
    

    X_train = None
    y_train = None
    X_test = None
    y_test = None
    
    if features_train is not None and target_train is not None and features_test is not None and target_test is not None:
        X_train = features_train
        y_train = target_train
        X_test = features_test
        y_test = target_test
    else:
        # Split the dataset into training and testing
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=test_size, random_state=42)


    # Split the training data into training and validation
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)

    # Save the predictions of each model
    predictions = []

    # Save the models
    base_models = []

    # draw violin plots for each performance metric(Accuracy, Sensitivity, Specificity, Precision ,F1-score, AUROC, AUPR) for the 9 bagging LR learners.
    # Save performance metrics for each base classifier
    base_metrics = {
        'accuracy': [],
        'sensitivity': [],
        'specificity': [],
        'precision': [],
        'f1': [],
        'auroc': [],
        'aupr': []
    }



    for i in range(n_base_classifiers):
        # Create an instance of the Logistic Regression model
        model = MyLogisticRegression(learning_rate=lr, num_iteration=num_iteration, regularization=regularization, lambda_=lambda_)
        # model = LogisticRegression()

        # Sample with replacement from 100% of the training data with resample function
        X_train_sampled, y_train_sampled = resample(X_train, y_train, replace=True, random_state=i, n_samples=len(X_train))

        # Train the model
        model.fit(X_train_sampled, y_train_sampled)

        # Save the model
        base_models.append(model)

        # Make predictions
        y_pred = model.predict(X_val)

        # Save the predictions
        predictions.append(y_pred)


    # Convert the list of predictions to a numpy array
    predictions = np.array(predictions)

    # Shape = (n_samples, n_base_classifiers)
    # Add these predictions as features to the validation set   
    X_val = np.hstack((X_val, predictions.T))
   

    # Train a meta-classifier on the validation set
    meta_classifier = MyLogisticRegression(learning_rate=lr, num_iteration=num_iteration, regularization=regularization, lambda_=lambda_)
    # meta_classifier = LogisticRegression()
    meta_classifier.fit(X_val, y_val)

    # Save the predictions of each model on the test set
    test_predictions = []
    test_predictions_proba = []

    for model in base_models:
        pred = model.predict(X_test)
        test_predictions.append(pred)

        # Save the prediction probabilities
        pred_proba = model.predict_proba(X_test)
        test_predictions_proba.append(pred_proba)

    
        accuracy, sensitivity, specificity, precision, f1, auroc, aupr = calculate_score(y_test, pred, pred_proba)

        
        base_metrics['accuracy'].append(accuracy)
        base_metrics['sensitivity'].append(sensitivity)
        base_metrics['specificity'].append(specificity)
        base_metrics['precision'].append(precision)
        base_metrics['f1'].append(f1)
        base_metrics['auroc'].append(auroc)
        base_metrics['aupr'].append(aupr)


    # Calculate the majority voting predictions
    test_predictions_proba_m = np.array(test_predictions_proba)
    test_predictions_proba_m = test_predictions_proba_m.mean(axis=0)
    
    test_predictions_m = np.array(test_predictions)
    test_predictions_m = test_predictions_m.T
    majority_voting_pred = []
    for i in range(test_predictions_m.shape[0]):
        majority_voting_pred.append(np.bincount(test_predictions_m[i]).argmax())
    accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV = calculate_score(y_test, majority_voting_pred, test_predictions_proba_m)


    
    
    # Convert the list of predictions to a numpy array
    test_predictions = np.array(test_predictions)

    # Shape = (n_samples, n_base_classifiers)
    # Add these predictions as features to the test set
    X_test = np.hstack((X_test, test_predictions.T))

    # Make predictions using the meta-classifier
    meta_classifier_pred = meta_classifier.predict(X_test)
    meta_classifier_pred_proba = meta_classifier.predict_proba(X_test)

    # Calculate the scores
    accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE = calculate_score(y_test, meta_classifier_pred, meta_classifier_pred_proba)

    return accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE, accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV, base_metrics
    

In [40]:
# Function for table creation
def create_table(data):
    headers = ["Model", "Accuracy", "Sensitivity", "Specificity", "Precision", "F1", "AUROC", "AUPR"]
    table = tabulate(data, headers, tablefmt="pretty")
    print(table)

In [41]:
def run_dataset1_churn(path = 'WA_Fn-UseC_-Telco-Customer-Churn.csv' , top_n_features=100, val_size=0.2, test_size=0.2, n_base_classifiers=9, lr=0.01, num_iteration=10000, regularization='l2', lambda_=0.01):
    dataframe = load_dataset(path)

    # Replace all the ' ' with np.nan
    dataframe.replace(' ', np.nan, inplace=True)

    # Convert the 'TotalCharges' column to numeric
    dataframe['TotalCharges'] = pd.to_numeric(dataframe['TotalCharges'], errors='coerce')

    # Drop the 'customerID' column, cuz it is not useful
    dataframe.drop('customerID', axis=1, inplace=True)

    # preprocess the dataset: replace null values, encode categorical columns, scale the features
    features, target = preprocess_dataset(dataframe, target_column='Churn', scaling_strategy='Standard', top_n_features=top_n_features)


    accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE, accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV, base_metrics = stacking_ensemble(features, target, val_size=val_size, test_size = test_size, n_base_classifiers=n_base_classifiers, regularization=regularization, lambda_=lambda_, lr=lr, num_iteration=num_iteration)

    # Calculate mean and standard deviation for each metric
    mean_std = calculate_mean_std(base_metrics)


    # Format the data so that it shows as 'mean +- std' in the table
    data = [
        ["Logistic Regression", f"{mean_std['accuracy']['mean']:f} ± {mean_std['accuracy']['std']:f}",  f"{mean_std['sensitivity']['mean']:f} ± {mean_std['sensitivity']['std']:f}", f"{mean_std['specificity']['mean']:f} ± {mean_std['specificity']['std']:f}", f"{mean_std['precision']['mean']:f} ± {mean_std['precision']['std']:f}", f"{mean_std['f1']['mean']:f} ± {mean_std['f1']['std']:f}", f"{mean_std['auroc']['mean']:f} ± {mean_std['auroc']['std']:f}", f"{mean_std['aupr']['mean']:f} ± {mean_std['aupr']['std']:f}"],
        ["Majority Voting", accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV],
        ["Stacking Ensemble", accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE]
    ]

    create_table(data)

    # Draw violin plots
    draw_violin_plots(base_metrics)

In [42]:
def run_dataset2_adult(path_data = 'adult.data', path_test = 'adult.test', top_n_features=200, val_size=0.2, test_size=0.2, n_base_classifiers=9, lr=0.01, num_iteration=10000, regularization='l2', lambda_=0.01):
    # load adult.data dataset
    headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
    dataframe_train = pd.read_csv(path_data, names=headers, engine='python')

    # load adult.test dataset
    dataframe_test = pd.read_csv(path_test, names=headers, engine='python', skiprows=1)


    # Replace all the ' ?' with np.nan
    dataframe_train.replace(' ?', np.nan, inplace=True)
    dataframe_test.replace(' ?', np.nan, inplace=True)


    # replace <=50K. with <=50K and >50K. with >50K
    dataframe_test['income'] = dataframe_test['income'].replace({' <=50K.': ' <=50K', ' >50K.': ' >50K'})


    train_len = len(dataframe_train)
    test_len = len(dataframe_test)
    ratio = train_len / (train_len + test_len)


    dataframe = pd.concat([dataframe_train, dataframe_test])

    #preprocess the dataset: replace null values, encode categorical columns, scale the features and select top 200 features
    features, target = preprocess_dataset(dataframe, target_column='income', scaling_strategy='Standard', top_n_features=top_n_features)

    # Split the features and target back into training and testing data according to the ratio
    train_len = int(ratio * len(features))
    features_train = features[:train_len]
    target_train = target[:train_len]

    features_test = features[train_len:]
    target_test = target[train_len:]

    # Run the stacking ensemble
    accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE, accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV, base_metrics = stacking_ensemble(features, target, val_size=val_size, test_size=test_size, n_base_classifiers=n_base_classifiers, regularization=regularization, lambda_=lambda_, features_train=features_train, target_train=target_train, features_test=features_test, target_test=target_test, num_iteration=num_iteration, lr=lr)


    # Calculate mean and standard deviation for each metric
    mean_std = calculate_mean_std(base_metrics)



    data = [
        ["Logistic Regression", f"{mean_std['accuracy']['mean']:f} ± {mean_std['accuracy']['std']:f}",  f"{mean_std['sensitivity']['mean']:f} ± {mean_std['sensitivity']['std']:f}", f"{mean_std['specificity']['mean']:f} ± {mean_std['specificity']['std']:f}", f"{mean_std['precision']['mean']:f} ± {mean_std['precision']['std']:f}", f"{mean_std['f1']['mean']:f} ± {mean_std['f1']['std']:f}", f"{mean_std['auroc']['mean']:f} ± {mean_std['auroc']['std']:f}", f"{mean_std['aupr']['mean']:f} ± {mean_std['aupr']['std']:f}"],
        ["Majority Voting", accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV],
        ["Stacking Ensemble", accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE]
    ]

    create_table(data)

    # Draw violin plots
    draw_violin_plots(base_metrics)



In [43]:
def run_dataset3_creditcardfraud(path='creditcardfraud.csv',top_n_features=100, val_size=0.2, test_size=0.2, n_base_classifiers=9, lr=0.01, num_iteration=10000, regularization='l2', lambda_=0.01):
    dataframe = load_dataset(path)
    # count the number of '1' and '0' in the 'Class' column
    print(dataframe['Class'].value_counts())
    print()

    # (randomly selected 20000 negative samples + all positive samples)
    dataframe = dataframe.sample(frac=1, random_state=42)
    negative_samples = dataframe[dataframe['Class'] == 0].head(20000)
    positive_samples = dataframe[dataframe['Class'] == 1]
    dataframe = pd.concat([negative_samples, positive_samples])

    # preprocess the dataset: replace null values, encode categorical columns, scale the features, select top n features based on correlation
    features, target = preprocess_dataset(dataframe, target_column='Class', scaling_strategy='Standard', top_n_features=top_n_features, columns_to_scale=['Time', 'Amount'])

    accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE, accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV, base_metrics = stacking_ensemble(features, target, val_size=val_size, test_size=test_size, n_base_classifiers=n_base_classifiers, regularization=regularization, lambda_=lambda_, num_iteration=num_iteration, lr=lr)

    # Calculate mean and standard deviation for each metric
    mean_std = calculate_mean_std(base_metrics)


    # Format the data so that it shows as 'mean +- std' in the table
    data = [
        ["Logistic Regression", f"{mean_std['accuracy']['mean']:f} ± {mean_std['accuracy']['std']:f}",  f"{mean_std['sensitivity']['mean']:f} ± {mean_std['sensitivity']['std']:f}", f"{mean_std['specificity']['mean']:f} ± {mean_std['specificity']['std']:f}", f"{mean_std['precision']['mean']:f} ± {mean_std['precision']['std']:f}", f"{mean_std['f1']['mean']:f} ± {mean_std['f1']['std']:f}", f"{mean_std['auroc']['mean']:f} ± {mean_std['auroc']['std']:f}", f"{mean_std['aupr']['mean']:f} ± {mean_std['aupr']['std']:f}"],
        ["Majority Voting", accuracy_MV, sensitivity_MV, specificity_MV, precision_MV, f1_MV, auroc_MV, aupr_MV],
        ["Stacking Ensemble", accuracy_SE, sensitivity_SE, specificity_SE, precision_SE, f1_SE, auroc_SE, aupr_SE]
    ]

    create_table(data)

    # Draw violin plots
    draw_violin_plots(base_metrics)

<h3><i>Telco Customer Churn (Dataset-1) </i> </h3>
<b><i> Comment out the later portion to run on dataset-1: Churn data </i></b><br>
Link: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

In [ ]:
run_dataset1_churn(
    path = 'WA_Fn-UseC_-Telco-Customer-Churn.csv',
    top_n_features=20,
    val_size=0.2,
    test_size=0.2,
    n_base_classifiers=9,
    lr=0.01,
    num_iteration=10000,
    regularization='l2',
    lambda_=0.01
)

<h3><i>Adult (Dataset-2): Predict whether income exceeds $50K/yr based on census data </i> </h3>

<b><i> Comment out the later portion to run on dataset-2: Adult data </i></b><br>
Link: https://archive.ics.uci.edu/dataset/2/adult

In [ ]:
run_dataset2_adult(
    path_data = 'adult.data',
    path_test = 'adult.test',
    top_n_features=20,
    n_base_classifiers=9,
    lr=0.01,
    num_iteration=10000,
    regularization='l2',
    lambda_=0.01
)

<h3><i>Credit Card Fraud Detection (Dataset-3) </i> </h3>

<b><i> Comment out the later portion to run on dataset-3: Credit Card Fraud Detection </i></b><br>
Link: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

In [ ]:
run_dataset3_creditcardfraud(
    path='creditcardfraud.csv',
    top_n_features=20,
    val_size=0.2,
    test_size=0.2,
    n_base_classifiers=9,
    lr=0.01,
    num_iteration=10000,
    regularization='l2',
    lambda_=0.01
)